In [2]:
import json
import folium
from folium.plugins import HeatMap
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
import io
import os
import imageio
from nextbike_functions import *

In [3]:
def two_hours_forward(timestamp_string: str):
    date, time = timestamp_string.split(' ')
    year, month, day = date.split('-')
    hour, minute, second = time.split(':')
    date = datetime.datetime(int(year), int(month), int(day), int(hour), int(minute), int(second))
    date += datetime.timedelta(hours=2)
    return date

For some reason, function defined above doesn't work, when declared in and imported from external file

In [4]:
data = []
timestamp_dictionary = dict()
filenames = []

i = 0
# Extracting stations info for stations in Warsaw
with open('json_files/places_and_bikes.json', 'r', encoding='utf8') as file:
    for line in file:
        data_piece = json.loads(line)
        if data_piece['city_uid'] == 812:
            timestamp = str(two_hours_forward(data_piece['timestamp']['$date'].replace('T', ' ')[:-1])).replace(':', '.')
            if not timestamp in timestamp_dictionary.keys():
                timestamp_dictionary[timestamp] = []
            timestamp_dictionary[timestamp].append(data_piece)


In [5]:
freestanding_coordinates_data = {}
for timestamp, data in timestamp_dictionary.items():
    freestanding_coordinates_data[timestamp] = []
    for station in data:
        if station['bike']:
            freestanding_coordinates_data[timestamp].append((station['lat'], station['lng']))

In [67]:
used_font = ImageFont.truetype('arial.ttf', 30)
target_folder = 'heatmaps/'

if not os.path.exists(target_folder):
    os.mkdir(target_folder)

for timestamp, freestanding_coordinates in freestanding_coordinates_data.items():
    m = folium.Map(location=[52.2265, 21.0196], zoom_start=14)
    HeatMap(freestanding_coordinates).add_to(m)
    filename = target_folder + timestamp + '.png'
    img_data = m._to_png(5)
    img = Image.open(io.BytesIO(img_data))
    
    I1 = ImageDraw.Draw(img)
    I1.text((img.size[0]-350, img.size[1]-80), timestamp, font = used_font, fill = (0, 0, 0))
    
    img.save(filename)
    filenames.append(filename)
    print(os.path.basename(filename), "done")

images = []
for filename in filenames:
    images.append(imageio.v2.imread(filename))
imageio.mimsave('movie.gif', images)

2024-06-28 15.05.00.png done
2024-06-28 15.10.00.png done
2024-06-28 15.15.00.png done
2024-06-28 15.20.00.png done
2024-06-28 15.25.00.png done
2024-06-28 15.30.00.png done
2024-06-28 15.35.00.png done


In [7]:
freestanding_coordinates_data

{'2024-06-28 15.05.00': [(52.246516, 21.13296),
  (52.436644, 21.02296),
  (52.246516, 21.13302),
  (52.185889, 20.850884),
  (52.225293, 20.904671),
  (52.246329, 21.065222),
  (52.202342, 21.115727),
  (52.140756, 20.846378),
  (52.27012, 20.991527),
  (52.20624, 21.06342),
  (52.290249, 21.029773),
  (52.306111, 21.081609),
  (52.207502, 21.151067),
  (52.251547, 21.01588),
  (52.264324, 21.064967),
  (52.116804, 21.242669),
  (52.176004, 20.81418),
  (52.31876, 20.966998),
  (52.193871, 21.024687),
  (52.190342, 21.071371),
  (52.25336, 21.04074),
  (52.299933, 20.939436),
  (52.193667, 20.965222),
  (52.161067, 21.210202),
  (52.270387, 20.991316),
  (52.239458, 20.898036),
  (52.232249, 20.929084),
  (52.195791, 20.863651),
  (52.199693, 21.120158),
  (52.296476, 20.930367),
  (52.233089, 21.011893),
  (52.224324, 21.065911),
  (52.235253, 20.976289),
  (52.264738, 21.038082),
  (52.269244, 21.002547),
  (52.243676, 20.944267),
  (52.072836, 20.877331),
  (52.188982, 21.065107),
